In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows

In [2]:
# Load dataset
df = pd.read_csv('3_dataset_dtypes_corrected.csv', index_col=False)

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PolynomialFeatures

In [5]:
# %% encode dataset để train
def encode(df):
    label_encoders = {}
    for column in df.columns:
        le = preprocessing.LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le
    
    return df, label_encoders
    
# %% decode
def decode(df, label_encoders):
    for column in df.columns:
        df[column] = label_encoders[column].inverse_transform(df[column])
    
    return df

# Multivariate Linear Regression
def linearReg(*args, df):    
    model = LinearRegression()
    
    df2 = df.copy()
    
    model.fit(df2[[*args]], df2['used_price'])

    return model

# Polynomial Regression
def polyReg(*args, df, deg=2):
    poly = PolynomialFeatures(deg)
    df2 = df.copy()[[*args]]
    print(df2.shape)
    y = df.copy()['used_price']
    
    df2 = poly.fit_transform(df2)
    print(df2.shape)
    
    model = LinearRegression()
    model.fit(df2, y)
    
    return model

In [6]:
# encode
df2 = df.copy()
df2_encoded, label_encoders = encode(df2)

# fill na bằng KNNImputer
imputer = KNNImputer(missing_values='nan', n_neighbors=10)
_ = imputer.fit_transform(df2_encoded)

# Build multivariate linear regression model
lr = linearReg(*df2_encoded.columns.drop('used_price'), df=df2_encoded)
# df2 = decode(df2_encoded, label_encoders)

In [7]:
lr.coef_
len(lr.coef_)

array([  0.91004139,   5.38892033,  -0.59657001,   7.11701458,
        10.90793218, -10.38286321,   0.2580827 ,  -1.90298916,
         7.70487317,  -6.71987263,   4.96186344,  -2.30740803,
        -5.38681758, -15.51960103, -19.87011295,  -0.72460677,
         1.61278129,  -1.99361768,   0.81498866,  -0.16109812,
        -0.18372763,  -0.14312347,   4.97661541,  34.64350175,
        31.0074749 ,  12.44896217,  -5.17346279,  -6.84159635,
        -7.69456911,  20.94843074,  -3.43301979, -29.38839437,
         1.1624377 ])

33

In [8]:
# Build polynomial regression model
polyModel = polyReg(*df2.columns.drop('used_price'), df=df2)
len(polyModel.coef_)

(301, 33)
(301, 595)


595